# Advanced Credit Card Fraud Detection using RNN

This notebook implements an advanced credit card fraud detection system using Recurrent Neural Networks (RNN) with TensorFlow. We'll use LSTM layers to capture sequential patterns in credit card transactions and build a robust fraud detection model.

In [7]:
!pip install "numpy<2"

Defaulting to user installation because normal site-packages is not writeable


## 1. Import Required Libraries

Let's import all the necessary libraries for our project:

In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import warnings
warnings.filterwarnings('ignore')


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "C:\ProgramData\anaconda3\Lib\site-pack

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: initialization failed

## 2. Data Loading and Preprocessing

Let's load the credit card fraud dataset and perform initial preprocessing:

In [ ]:
# Load the dataset
df = pd.read_csv('data/creditcard.csv')

# Check for missing values
print("Missing values:\n", df.isnull().sum())

# Display basic information about the dataset
print("\nDataset Info:")
print(df.info())

# Display class distribution
print("\nClass distribution:")
print(df['Class'].value_counts(normalize=True))

# Separate features and target
X = df.drop('Class', axis=1)
y = df['Class']

# Create sequences for RNN (using time steps of 3)
def create_sequences(X, y, time_steps=3):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# Create sequences
X_seq, y_seq = create_sequences(X_scaled, y)

print("\nSequence shape:", X_seq.shape)
print("Target shape:", y_seq.shape)

## 3. Exploratory Data Analysis

Let's analyze the data distribution and handle class imbalance:

In [ ]:
# Visualize class distribution
plt.figure(figsize=(8, 6))
sns.countplot(data=df, x='Class')
plt.title('Class Distribution (Before SMOTE)')
plt.show()

# Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_seq_reshaped = X_seq.reshape(X_seq.shape[0], -1)
X_resampled, y_resampled = smote.fit_resample(X_seq_reshaped, y_seq)
X_resampled = X_resampled.reshape(-1, X_seq.shape[1], X_seq.shape[2])

print("Original shape:", X_seq.shape)
print("Resampled shape:", X_resampled.shape)

# Visualize feature distributions
plt.figure(figsize=(15, 5))
for i in range(3):
    plt.subplot(1, 3, i+1)
    sns.histplot(data=df, x=f'V{i+1}', hue='Class', bins=50)
    plt.title(f'Distribution of V{i+1}')
plt.tight_layout()
plt.show()

## 4. Building RNN Model

Let's create our advanced RNN model using LSTM layers:

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Build the advanced RNN model with 6 hidden layers
model = Sequential([
    # 1st LSTM layer with tanh activation (default for LSTM)
    LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    Dropout(0.3),
    
    # 2nd LSTM layer with tanh activation
    LSTM(96, return_sequences=True),
    Dropout(0.3),
    
    # 3rd LSTM layer with tanh activation
    LSTM(64, return_sequences=True),
    Dropout(0.3),
    
    # 4th LSTM layer with tanh activation
    LSTM(48, return_sequences=True),
    Dropout(0.3),
    
    # 5th LSTM layer with tanh activation
    LSTM(32, return_sequences=True),
    Dropout(0.3),
    
    # 6th LSTM layer
    LSTM(24, return_sequences=False),
    Dropout(0.3),
    
    # Dense layers with different activation functions
    Dense(64, activation='relu'),  # ReLU activation
    Dense(48, activation='elu'),   # ELU activation
    Dense(32, activation='selu'),  # SELU activation
    Dense(24, activation='tanh'),  # Tanh activation
    Dense(16, activation='swish'), # Swish activation
    Dense(8, activation='sigmoid'), # Sigmoid activation
    
    # Output layer
    Dense(1, activation='sigmoid')
])

# Compile the model with binary crossentropy loss
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Binary classification loss function
              metrics=['accuracy', tf.keras.metrics.AUC(), 
                      tf.keras.metrics.Precision(), 
                      tf.keras.metrics.Recall()])

# Display model summary
model.summary()

## 5. Model Training

Let's train our model with early stopping to prevent overfitting:

In [ ]:
# Define callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=0.00001
)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=100,  # Increased epochs for complex model
    batch_size=64,  # Adjusted batch size
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr]
)

# Plot training history
plt.figure(figsize=(15, 5))

# Plot loss
plt.subplot(1, 3, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot accuracy
plt.subplot(1, 3, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot AUC
plt.subplot(1, 3, 3)
plt.plot(history.history['auc'], label='Training AUC')
plt.plot(history.history['val_auc'], label='Validation AUC')
plt.title('Model AUC')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.legend()

plt.tight_layout()
plt.show()

# Plot additional metrics
plt.figure(figsize=(12, 4))

# Plot Precision
plt.subplot(1, 2, 1)
plt.plot(history.history['precision'], label='Training Precision')
plt.plot(history.history['val_precision'], label='Validation Precision')
plt.title('Model Precision')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.legend()

# Plot Recall
plt.subplot(1, 2, 2)
plt.plot(history.history['recall'], label='Training Recall')
plt.plot(history.history['val_recall'], label='Validation Recall')
plt.title('Model Recall')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.legend()

plt.tight_layout()
plt.show()

## 6. Model Evaluation

Let's evaluate our model's performance using various metrics:

In [ ]:
# Make predictions
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_classes))

# Create confusion matrix
cm = confusion_matrix(y_test, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Calculate ROC curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

## 7. Save and Load Model

Let's save our trained model for future use:

In [ ]:
# Save the model and scaler
model.save('fraud_detection_model.h5')
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("Model and scaler saved successfully!")

# Function to load the model and scaler
def load_model_and_scaler():
    loaded_model = tf.keras.models.load_model('fraud_detection_model.h5')
    with open('scaler.pkl', 'rb') as f:
        loaded_scaler = pickle.load(f)
    return loaded_model, loaded_scaler

# Test loading
loaded_model, loaded_scaler = load_model_and_scaler()
print("Model and scaler loaded successfully!")

## 8. Real-time Prediction

Let's create a function for real-time fraud detection:

In [ ]:
def predict_fraud(transaction_data, model, scaler, time_steps=3):
    """
    Predict if a transaction is fraudulent
    
    Parameters:
    transaction_data: numpy array of shape (30,) containing the transaction features
    model: loaded keras model
    scaler: loaded StandardScaler
    time_steps: number of time steps used in training
    
    Returns:
    probability of fraud, prediction (0: legitimate, 1: fraud)
    """
    # Scale the transaction
    scaled_transaction = scaler.transform(transaction_data.reshape(1, -1))
    
    # Create sequence (assuming we have previous transactions)
    # For demo, we'll just repeat the same transaction
    sequence = np.repeat(scaled_transaction, time_steps, axis=0)
    sequence = sequence.reshape(1, time_steps, -1)
    
    # Make prediction
    fraud_probability = model.predict(sequence)[0][0]
    fraud_prediction = 1 if fraud_probability > 0.5 else 0
    
    return fraud_probability, fraud_prediction

# Test the function with a sample transaction
sample_transaction = X.iloc[0].values
prob, pred = predict_fraud(sample_transaction, loaded_model, loaded_scaler)

print(f"Fraud Probability: {prob:.2%}")
print(f"Prediction: {'Fraudulent' if pred == 1 else 'Legitimate'} Transaction")